In [1]:
import sys

sys.path.append('/user/ms5941/NLP')

from config import *

import gensim

import glob
import os
import re

import numpy as np
import pandas as pd

import json

/user/ms5941/.local/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

from gensim.models import ldaseqmodel

In [3]:
THEME = 'Inflation'

In [5]:
# Load dictionary and corpus
dictionary_all = gensim.corpora.Dictionary.load(TEMP_PATH + '/%s/%s_less_restricted.dict' % (THEME, THEME))

2021-04-25 22:34:06,072 : INFO : loading Dictionary object from /work/ms5941/NLP/Temp/Inflation/Inflation_less_restricted.dict
2021-04-25 22:34:06,078 : INFO : Dictionary lifecycle event {'fname': '/work/ms5941/NLP/Temp/Inflation/Inflation_less_restricted.dict', 'datetime': '2021-04-25T22:34:06.077834', 'gensim': '4.0.1', 'python': '3.7.7 (default, May  7 2020, 21:25:33) \n[GCC 7.3.0]', 'platform': 'Linux-4.19.0-16-amd64-x86_64-with-debian-10.9', 'event': 'loaded'}


In [ ]:
# 1. Look for ids of related words to theme in the dictionary. This will be a list of ids for each year.
# 

In [6]:
words = pd.read_csv(TEMP_PATH + '/%s_words.csv' % THEME.lower(), index_col=0)

In [7]:
year_name_list = dict()
year_id_list = dict()
for col in words.columns:
    reqd_words = list(words[col].values) + [THEME.lower()]
    reqd_ids = [k for k,v in dictionary_all.items() if v in reqd_words]
    year_name_list[col] = [dictionary_all[i] for i in reqd_ids]
    year_id_list[col] = reqd_ids

In [8]:
year_name_list

{'1996': ['inflation',
  'gdp',
  'unemployment',
  'pressures',
  'wage',
  'cpi',
  'economy'],
 '1997': ['inflation', 'unemployment', 'pressures', 'wage', 'cpi', 'economy'],
 '1998': ['inflation', 'gdp', 'unemployment', 'wage', 'cpi', 'economy'],
 '1999': ['inflation', 'gdp', 'wage', 'cpi', 'economy'],
 '2000': ['inflation', 'gdp', 'pressures', 'wage', 'cpi', 'economy'],
 '2001': ['inflation',
  'gdp',
  'unemployment',
  'cpi',
  'ecb',
  'growth',
  'economy'],
 '2002': ['inflation', 'gdp', 'cpi', 'ecb', 'economy'],
 '2003': ['inflation',
  'fed',
  'gdp',
  'payrolls',
  'cpi',
  'policymakers',
  'ecb',
  'economy'],
 '2004': ['inflation',
  'fed',
  'gdp',
  'payrolls',
  'cpi',
  'greenspan',
  'bernanke'],
 '2005': ['easing', 'inflation', 'fed', 'gdp', 'payrolls', 'cpi', 'greenspan'],
 '2006': ['inflation', 'fed', 'gdp', 'payrolls', 'cpi', 'bernanke'],
 '2007': ['easing',
  'inflation',
  'fed',
  'gdp',
  'recession',
  'cpi',
  'policymakers',
  'tightening'],
 '2008': ['ea

In [10]:
years = [str(year) for year in range(START_YEAR, END_YEAR + 1)]

In [11]:
good_words_count_per_year = dict()
for year in years:
    good_words_count_per_year[year] = []
    with open(TOKENIZED_ARTICLES_PATH % (THEME, THEME, year)) as f:
        a = json.load(f)
        for article in a:
            good_words_count_per_year[year].append(len([word for word in article if word in year_name_list[year]]))
    print(TOKENIZED_ARTICLES_PATH % (THEME, THEME, year), 'done!')

/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_1996.json done!
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_1997.json done!
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_1998.json done!
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_1999.json done!
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_2000.json done!
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_2001.json done!
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_2002.json done!
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_2003.json done!
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_2004.json done!
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_2005.json done!
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_2006.json done!
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_2007.json done!
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_2008.json done!

In [12]:
theme = 'Inflation'

all_tok_articles = []
yearly_slices = []
for year in years:
    # Get the 1% threshold count (the top 1% of most relevant articles)
    x = good_words_count_per_year[year]
    x.sort()
    threshold = x[-int(len(x)*0.01):][0]
    year_article_count = 0
    with open(TOKENIZED_ARTICLES_PATH % (theme, theme, year)) as f:
        all_articles = json.load(f)
        article_index = 0
        for articles in all_articles:
            if good_words_count_per_year[year][article_index] >= threshold:
                all_tok_articles.append(articles)
                year_article_count += 1    
            article_index += 1
    yearly_slices.append(year_article_count)
    print(TOKENIZED_ARTICLES_PATH % (theme, theme, year), 'done!')
    print('Count for the year:', year_article_count, 'Vs original:', len(all_articles))

# dictionary_all = gensim.corpora.Dictionary.load(TEMP_PATH + '/%s/%s_less_restricted.dict' % (theme, theme))


/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_1996.json done!
Count for the year: 335 Vs original: 30955
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_1997.json done!
Count for the year: 480 Vs original: 38966
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_1998.json done!
Count for the year: 420 Vs original: 40768
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_1999.json done!
Count for the year: 774 Vs original: 63481
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_2000.json done!
Count for the year: 718 Vs original: 60976
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_2001.json done!
Count for the year: 924 Vs original: 74314
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_2002.json done!
Count for the year: 504 Vs original: 39424
/work/ms5941/NLP/Temp/Inflation/Inflation_Articles_Tokenized_2003.json done!
Count for the year: 1391 Vs original: 132201
/work/ms5941/NLP/Temp/Inflation/Inflat

In [13]:
dictionary_all = gensim.corpora.Dictionary.load(TEMP_PATH + '/%s/%s_less_restricted.dict' % (theme, theme))

class MyCorpus:
    def __iter__(self):
        for doc in all_tok_articles:
            yield dictionary_all.doc2bow(doc)

corpus_memory_friendly = MyCorpus()  # doesn't load the corpus into memory!
print(corpus_memory_friendly)

gensim.corpora.MmCorpus.serialize(TEMP_PATH + '/%s/%s_less_restricted.mm' % (theme, theme), corpus_memory_friendly)


2021-04-25 22:46:20,774 : INFO : loading Dictionary object from /work/ms5941/NLP/Temp/Inflation/Inflation_less_restricted.dict
2021-04-25 22:46:20,779 : INFO : Dictionary lifecycle event {'fname': '/work/ms5941/NLP/Temp/Inflation/Inflation_less_restricted.dict', 'datetime': '2021-04-25T22:46:20.779496', 'gensim': '4.0.1', 'python': '3.7.7 (default, May  7 2020, 21:25:33) \n[GCC 7.3.0]', 'platform': 'Linux-4.19.0-16-amd64-x86_64-with-debian-10.9', 'event': 'loaded'}
2021-04-25 22:46:20,781 : INFO : storing corpus in Matrix Market format to /work/ms5941/NLP/Temp/Inflation/Inflation_less_restricted.mm
2021-04-25 22:46:20,786 : INFO : saving sparse matrix to /work/ms5941/NLP/Temp/Inflation/Inflation_less_restricted.mm
2021-04-25 22:46:20,787 : INFO : PROGRESS: saving document #0


2021-04-25 22:46:21,124 : INFO : PROGRESS: saving document #1000
2021-04-25 22:46:21,499 : INFO : PROGRESS: saving document #2000
2021-04-25 22:46:21,907 : INFO : PROGRESS: saving document #3000
2021-04-25 22:46:22,388 : INFO : PROGRESS: saving document #4000
2021-04-25 22:46:22,738 : INFO : PROGRESS: saving document #5000
2021-04-25 22:46:23,109 : INFO : PROGRESS: saving document #6000
2021-04-25 22:46:23,511 : INFO : PROGRESS: saving document #7000
2021-04-25 22:46:23,913 : INFO : PROGRESS: saving document #8000
2021-04-25 22:46:24,343 : INFO : PROGRESS: saving document #9000
2021-04-25 22:46:24,765 : INFO : PROGRESS: saving document #10000
2021-04-25 22:46:25,175 : INFO : PROGRESS: saving document #11000
2021-04-25 22:46:25,627 : INFO : PROGRESS: saving document #12000
2021-04-25 22:46:26,067 : INFO : PROGRESS: saving document #13000
2021-04-25 22:46:26,544 : INFO : PROGRESS: saving document #14000
2021-04-25 22:46:27,001 : INFO : PROGRESS: saving document #15000
2021-04-25 22:46:27

In [14]:
sum(yearly_slices)

29940

In [5]:
with open(TEMP_PATH + '/%s/%s_yearly_slices.txt' % (THEME, THEME), 'r') as f:
    yearly_slices = json.load(f)

In [6]:
dictionary_all = gensim.corpora.Dictionary.load(TEMP_PATH + '/%s/%s_less_restricted.dict' % (THEME, THEME))
corpus_all = gensim.corpora.MmCorpus(TEMP_PATH + '/%s/%s_less_restricted.mm' % (THEME, THEME))


2021-04-26 17:17:52,968 : INFO : loading Dictionary object from /work/ms5941/NLP/Temp/Inflation/Inflation_less_restricted.dict
2021-04-26 17:17:52,972 : INFO : Dictionary lifecycle event {'fname': '/work/ms5941/NLP/Temp/Inflation/Inflation_less_restricted.dict', 'datetime': '2021-04-26T17:17:52.972198', 'gensim': '4.0.1', 'python': '3.7.7 (default, May  7 2020, 21:25:33) \n[GCC 7.3.0]', 'platform': 'Linux-4.19.0-16-amd64-x86_64-with-debian-10.9', 'event': 'loaded'}
2021-04-26 17:17:52,976 : INFO : loaded corpus index from /work/ms5941/NLP/Temp/Inflation/Inflation_less_restricted.mm.index
2021-04-26 17:17:52,977 : INFO : initializing cython corpus reader from /work/ms5941/NLP/Temp/Inflation/Inflation_less_restricted.mm
2021-04-26 17:17:52,979 : INFO : accepted corpus with 29940 documents, 1623 features, 3903782 non-zero entries


In [ ]:
ldaseq = ldaseqmodel.LdaSeqModel(corpus=corpus_all, 
                                 id2word=dictionary_all, 
                                 time_slice=yearly_slices,
                                 num_topics=10,
                                 em_min_iter=1,
                                 em_max_iter=4)

2021-04-26 17:18:58,640 : INFO : using symmetric eta at 0.1
2021-04-26 17:18:58,642 : INFO : using serial LDA version on this node
2021-04-26 17:18:58,645 : INFO : running online (multi-pass) LDA training, 10 topics, 10 passes over the supplied corpus of 29940 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2021-04-26 17:18:59,129 : INFO : PROGRESS: pass 0, at document #2000/29940
2021-04-26 17:19:04,638 : INFO : merging changes from 2000 documents into a model of 29940 documents
2021-04-26 17:19:04,648 : INFO : topic #0 (0.010): 0.008*"trading" + 0.005*"end" + 0.005*"euro" + 0.005*"time" + 0.005*"currency" + 0.004*"rose" + 0.004*"around" + 0.004*"close" + 0.004*"low" + 0.004*"record"
2021-04-26 17:19:04,649 : INFO : topic #6 (0.010): 0.008*"trading" + 0.006*"time" + 0.005*"around" + 0.005*"strong" + 0.004*"early" + 0.004*"still" + 0.004*"price" + 0.004*"washington" + 0.004*"end" + 

In [ ]:
ldaseq

In [ ]:
ldaseq.save(TEMP_PATH + '/%s/%s_seqLDA_10topics_yearly' %(THEME, THEME))

In [13]:
ldaseq.print_topics()

[[('asia', 0.11994923264326991),
  ('asian', 0.0697415123559616),
  ('gold', 0.03460714040468794),
  ('kong', 0.02807702484827421),
  ('china', 0.028031049642759223),
  ('hong', 0.02698856078712894),
  ('singapore', 0.021979139159210587),
  ('europe', 0.02174957484390464),
  ('record', 0.01299782779701541),
  ('says', 0.012895293251154323),
  ('crisis', 0.009867505376286783),
  ('metals', 0.009447741456457421),
  ('india', 0.009382043453496554),
  ('back', 0.008925651507687577),
  ('next', 0.008627656060710942),
  ('indian', 0.00844176297132092),
  ('fund', 0.008227267387854564),
  ('impact', 0.007641328893371286),
  ('biggest', 0.007581695620153075),
  ('precious', 0.007241046413286706)],
 [('mergers', 0.09742732352273752),
  ('acquisitions', 0.070949059827724),
  ('mln', 0.015376436526145584),
  ('phone', 0.014252242300853082),
  ('headlines', 0.014158120777784592),
  ('breaking', 0.01063083869989524),
  ('plus', 0.010507329115696443),
  ('screen', 0.01029822024908081),
  ('sports', 